# Advent of Code


In [23]:
import numpy as np
import re
import pandas as pd
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt4Agg


### Day 1: Not Quite Lisp

In [220]:
with open('./data/input1.txt', 'r') as f:
    data = f.next()
ups_and_downs = np.array([1 if x=='(' else -1 for x in data])
print ups_and_downs.sum()
print np.where(ups_and_downs.cumsum()==-1)[0][0] + 1

74
1795


### Day 2: I Was Told There Would Be No Math

In [221]:
with open('./data/input2.txt', 'r') as f:
    dims = np.array([map(str.strip, spec.split('x')) for spec in f], dtype=int)
l, w, h = dims.T
side_areas = np.array([l*w, w*h, h*l])
surf_areas = 2 * side_areas.sum(0)
print (surf_areas + side_areas.min(0)).sum()
print sum(((2 * sum(sorted(spec)[:2])) + spec.prod() for spec in dims))

1586300
3737498


### Day 3: Perfectly Spherical Houses in a Vacuum

In [222]:
with open('./data/input3.txt', 'r') as f:
    data = f.next()
translation_map = {'^':(0,1), '>':(1,0), 'v':(0,-1), '<':(-1,0)}
# don't forget to include the starting location (0,0) twice, once for Santa and once for Robo-Santa!
translations = np.array([(0,0), (0,0)] + [translation_map[d] for d in data])
santa, robosanta = translations[::2], translations[1::2]
def unique_locations(translations):
    path = translations.cumsum(0)
    return set(map(tuple, path))

print len(unique_locations(translations))
print len(set.union(unique_locations(santa), unique_locations(robosanta)))

2081
2341


### Day 4: The Ideal Stocking Stuffer

In [223]:
import hashlib as hl

def find_hash(key, min_zeros):
    for i in itertools.count():
        h = hl.md5(key + str(i))
        if h.hexdigest()[:min_zeros] == '0'*min_zeros:
            break
    return i

print find_hash('ckczppom', 5)
print find_hash('ckczppom', 6)

117946
3938038


### Day 5: Doesn't He Have Intern-Elves For This?

In [224]:
with open('./data/input5.txt', 'r') as f:
    strings = [s.strip() for s in f]

vowels_cond = np.array(map(lambda x: [c in 'aeiou' for c in x], strings)).sum(1) > 2
repeat_cond = np.array([len(re.findall('(\\w)\\1', x)) for x in strings]) > 0
forbid_cond = np.array(map(lambda x: [s in x for s in ('ab', 'cd', 'pq', 'xy')], strings)).sum(1) == 0
repeat_cond2 = np.array([len(re.findall('(\\w{2}).*\\1', x)) for x in strings]) > 0
repeat_cond3 = np.array([len(re.findall('(\\w).\\1', x)) for x in strings]) > 0

print (forbid_cond & repeat_cond & vowels_cond).sum()
print (repeat_cond2 & repeat_cond3).sum()

258
53


### Day 6: Probably a Fire Hazard

In [225]:
lights = np.zeros((1000, 1000), dtype=bool)
display = np.zeros((1000, 1000), dtype=int)
with open('./data/input6.txt', 'r') as f:
    coords = np.array([re.findall('(.*) (\d+),(\d+).*?(\d+),(\d+)', r)[0] for r in f])

commands, (x1, y1, x2, y2) = coords.T[0], coords.T[1:].astype(int)
for i, c in enumerate(commands):
    if c == 'turn on':
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = True
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] += 1
    elif c == 'turn off':
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = False
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] -= 1
    else:
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = ~lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1]
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] += 2
    display = np.maximum(0, display)

print lights.sum()
print np.maximum(0, display).sum()

377891
14110788


### Day 7: Some Assembly Required

In [226]:
d = {'AND':'&', 'OR':'|', 'RSHIFT':'>>', 'LSHIFT':'<<', 'NOT':'~'}
bitwise = re.compile(r'(' + '|'.join(d.keys()) + r')')

with open('./data/input7.txt', 'r') as f:
    data = f.read().replace('if', 'iff').replace('as', 'ass').replace('in', 'inn').replace('is', 'iss').split('\n')[:-1]

data = map(lambda x: bitwise.sub(lambda y: d[y.group()], x), data)  # replace by bit-wise operators
data = map(lambda x: re.findall('(.+) -> (.+)', x)[0], data)
circuit = {x[1]: x[1] + ' = ' + x[0] for x in data} # 

def run_circuit(circuit, out):
    '''Not efficient: Runs in O(N**2)'''
    for _ in range(len(circuit)):
        for wire in circuit:
            try: exec(circuit[wire])
            except: pass
    return eval(out)

a = run_circuit(circuit, 'a')
circuit['b'] = 'b = %d' % a  # override b signal
print a
print run_circuit(circuit, 'a')  # run updated circuit

16076
2797


###  Day 8: Matchsticks

In [227]:
with open('./data/input8.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

escapechars = re.compile('|'.join([r'\\', "'", '"']))

df = pd.DataFrame({'stringcode': data, 'string': map(eval, data),
                   'stringcodecode': map(lambda x: "'"+escapechars.sub(lambda y:'\\'+y.group(),x)+"'", data)
                  })
print (df.stringcode.map(len) - df.string.map(len)).sum()
print (df.stringcodecode.map(len) - df.stringcode.map(len)).sum()
df.head()

1333
2046


,string,stringcode,stringcodecode
0,"sjdivfriyaaqa�v""k""mpcu""yyu""en","""sjdivfriyaaqa\xd2v\""k\""mpcu\""yyu\""en""","'\""sjdivfriyaaqa\\xd2v\\\""k\\\""mpcu\\\""yyu\\\""..."
1,vcqc,"""vcqc""","'\""vcqc\""'"
2,"zbcwgmbpijcxu""yins""sfxn","""zbcwgmbpijcxu\""yins\""sfxn""","'\""zbcwgmbpijcxu\\\""yins\\\""sfxn\""'"
3,yumngprx,"""yumngprx""","'\""yumngprx\""'"
4,bbdj,"""bbdj""","'\""bbdj\""'"


### Day 9: All in a Single Night

In [228]:
with open('./data/input9.txt', 'r') as f:
    data = map(lambda x: x.split(' ')[::2], f.read().split('\n')[:-1])

df = pd.DataFrame(data, columns=['A', 'B', 'distance']).pivot('A', 'B', 'distance')

def dist(A, B):
    try: return int(df[A][B])
    except: return int(df[B][A])

min_tour_len = np.Inf
max_tour_len = 0
locations = df.columns | df.index
for tour in itertools.permutations(locations, len(locations)):
    tour_len = sum([dist(tour[j], tour[j+1]) for j in xrange(len(tour)-1)])
    min_tour_len = min(tour_len, min_tour_len)
    max_tour_len = max(tour_len, max_tour_len)

print min_tour_len
print max_tour_len

117
909


### Day 10: Elves Look, Elves Say

In [229]:
def look_and_say(N):
    new_N = ''
    last_char = N[0]
    count = 0
    for c in N:
        if c == last_char:
            count += 1
        else:
            new_N += str(count) + last_char
            last_char = c
            count = 1
    new_N += str(count) + last_char
    return new_N

N = '1113122113'
lengths = []
for _ in xrange(50):
    N = look_and_say(N)
    lengths.append(len(N))
print lengths[39]
print lengths[-1]

360154
5103798


### Day 11: Corporate Policy

In [230]:
def next_pwd(pwd):
    return next_pwd(pwd[:-1]) + 'a' if pwd[-1] == 'z' else pwd[:-1] + unichr(ord(pwd[-1]) + 1)

def check_for_staight(pwd):
    for i, c in enumerate(pwd[:-2]):
        if unichr(ord(c)+1) == pwd[i+1] and unichr(ord(c)+2) == pwd[i+2]:
            return True
    return False

def check_for_chars(pwd):
    return ~np.array([c in pwd for c in 'iol']).any()

def check_for_pairs(pwd):
    return len(re.findall('(\w)\\1', pwd)) > 1

def next_valid_pwd(pwd):
    for _ in itertools.count():
        if check_for_staight(pwd) and check_for_chars(pwd) and check_for_pairs(pwd):
            return pwd
        else:
            pwd = next_pwd(pwd)

pwd2 = next_valid_pwd('cqjxjnds')
print pwd2
print next_valid_pwd(next_pwd(pwd2))

cqjxxyzz
cqkaabcc


### Day 12: JSAbacusFramework.io

In [231]:
with open('./data/input12.txt', 'r') as f:
    data = eval(f.read())

def sum_nested_numerals(obj, exclude_color=None):
    s = 0
    if type(obj) is dict and exclude_color in obj.itervalues():
        return s
    elif type(obj) is dict:
        for key in obj:
            s += sum_nested_numerals(obj[key], exclude_color)
    elif type(obj) is list:
        for item in obj:
            s += sum_nested_numerals(item, exclude_color)
    elif type(obj) is int:
        s += obj
    return s

print sum_nested_numerals(data)
print sum_nested_numerals(data, exclude_color='red')

191164
87842


### Day 13: Knights of the Dinner Table

In [ ]:
with open('./data/input13.txt', 'r') as f:
    data = map(lambda x: np.array(x.replace('gain ', '+').replace('lose ', '-').split(' '))[[0,2,-1]], f.read().split('.\n')[:-1])

w = pd.DataFrame(np.array(zip(*data)).T, columns=['A','weight','B']).pivot('A','B','weight').fillna(0).astype(int)
w['You'] = 0
w.loc['You'] = 0

def max_seating_score(w):
    max_score = 0
    for seats in itertools.permutations(w.index, w.shape[0]):
        score = sum([w[seats[j]][seats[j+1]] for j in xrange(-1, len(seats)-1)])
        max_score = max(score, max_score)
    return max_score

print max_seating_score((w + w.T).iloc[:-1, :-1])
print max_seating_score(w + w.T)

### Day 14: Reindeer Olympics

In [195]:
with open('./data/input14.txt', 'r') as f:
    data = np.array(map(lambda x: np.array(x.split(' '))[[3,6,-2]], f.read().split('\n')[:-1]), dtype=int)

def state_in_time(speed, run_duration, rest_duration):
    cycles = int(np.ceil(float(time) / (run_duration + rest_duration)))
    return (([1]*run_duration + [0]*rest_duration) * cycles)

time = 2503
cum_distances = np.array(map(lambda x: np.cumsum(state_in_time(*x)[:time]) * x[0], data))

print max(cum_distances[:, -1])
print pd.Series(cum_distances.argmax(0)).value_counts().max()

2696
1084


### Day 15: Science for Hungry People

In [194]:
with open('./data/input15.txt', 'r') as f:
    # properties x ingredients
    properties = np.reshape(re.findall('-?\d+', f.read()), (4, 5)).astype(int).T

max_score = 0
max_score_lite = 0
for c in itertools.product(*[xrange(101)]*3):
    c = c[0], c[1], c[2], 100 - sum(c)
    if c[-1] >= 0:  # do the teaspoons add to 100?
        mix = np.maximum(0, np.dot(properties, c))
        max_score = max(max_score, mix[:-1].prod())
        if mix[-1] == 500:  # does this recipe yield 500 calories?
            max_score_lite = max(max_score_lite, mix[:-1].prod())

print max_score
print max_score_lite

13882464
11171160


### Day 16: Aunt Sue

In [191]:
df = pd.DataFrame(columns=['children', 'cats', 'samoyeds', 'pomeranians', 'akitas', 'vizslas',
                           'goldfish', 'trees', 'cars', 'perfumes'])
with open('./data/input16.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

aunt_signature = np.array([3, 7, 2, 3, 0, 0, 5, 3, 2, 1])
for r in tqdm(data):
    exec('df.loc[%s,:] = {\'%s\':%s, \'%s\':%s, \'%s\':%s}' % re.findall('(\d+): (.+): (\d+), (.+): (\d+), (.+): (\d+)', r)[0])
    
print df[df.apply(lambda x: sum(x == aunt_signature) == 3, axis=1)]
print df[df.apply(lambda x:
         sum(list(x[[0,2,4,5,8,9]]==aunt_signature[[0,2,4,5,8,9]]) +
         list(x[[1,7]]>aunt_signature[[1,7]])+
         list(x[[3,6]]<aunt_signature[[3,6]])) == 3,
         axis=1)]

    children cats samoyeds pomeranians akitas vizslas goldfish trees cars  \
373      NaN  NaN      NaN           3    NaN       0      NaN   NaN  NaN   

    perfumes  
373        1  
    children cats samoyeds pomeranians akitas vizslas goldfish trees cars  \
260      NaN  NaN        2         NaN    NaN       0        0   NaN  NaN   

    perfumes  
260      NaN  


### Day 17: No Such Thing as Too Much

In [11]:
with open('./data/input17.txt', 'r') as f:
    containers = np.array(f.read().split('\n')[:-1], dtype=int)

count = 0
combo_150 = []
for n in tqdm(xrange(1, len(containers) + 1)):
    for combo in itertools.combinations(containers, n):
        if sum(combo) == 150:
            count += 1
            combo_150.append(combo)

print count
print pd.Series(map(len, combo_150)).value_counts().iloc[-1]

4372
4


### Day 18: Like a GIF For Your Yard

In [ ]:
with open('./data/input18.txt', 'r') as f:
    lights = np.array(map(list, f.read().replace('#', '1').replace('.', '0').split('\n')[:-1]), dtype=int)
    lights2 = lights.copy()
    lights2[[0, 0, 99, 99], [0, 99, 0, 99]] = 1 
    
def next_state(lights):
    n = lights.shape[0]
    appended_lights = np.zeros((n + 2, n + 2))
    appended_lights[1:-1, 1:-1] = lights
    new_state = np.zeros_like(appended_lights)
    for i, j in itertools.product(xrange(1, n + 1), xrange(1, n + 1)):
        s = appended_lights[i - 1:i + 2, j - 1:j + 2].sum() - appended_lights[i, j]
        if appended_lights[i, j] == 1:
            new_state[i, j] = 0 if s not in [2, 3] else 1
        else:
            new_state[i, j] = 1 if s == 3 else 0
    return new_state[1:-1, 1:-1]

# plt.figure(figsize=(7,7))
for i in tqdm(range(100)):
    lights = next_state(lights)
    lights2 = next_state(lights2)
    lights2[[0, 0, 99, 99], [0, 99, 0, 99]] = 1
    
#     plt.imshow(lights, interpolation='nearest', cmap=plt.cm.Blues)
#     plt.axis('off')
#     plt.savefig('frame%03d.png' % i)
#     plt.cla()
# !convert -delay 5 *png animated.gif  # requires Imagemagick

print lights.sum()
print lights2.sum()

### Day 19: Medicine for Rudolph